# GRADCAM

In [1]:
# Make sure these are imported
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image as keras_image # Use alias to avoid conflict
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess_input # Or your model's specific preprocess
import cv2 # Optional, but often convenient for resizing. Install with: pip install opencv-python


In [ ]:
CONFIGURATION = {
    # Basic parameters
    "BATCH_SIZE": 32,
    "IM_SIZE": 224,
    "NUM_CLASSES": 4,
    "CLASS_NAMES": ["Fibres", "Nanowires", "Particles","Powder"],
}

In [ ]:
def preprocess_image(img_path, target_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"])):
    """Loads and preprocesses an image for the model."""
    img = keras_image.load_img(img_path, target_size=target_size)
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
    # Use the specific preprocessing function for your base model
    # For EfficientNet:
    processed_img = efficientnet_preprocess_input(img_array)
    # For ResNet50: from tensorflow.keras.applications.resnet50 import preprocess_input
    # processed_img = resnet50_preprocess_input(img_array)
    # If you only did rescaling during training (like 1./255):
    # processed_img = img_array / 255.0
    return processed_img

def deprocess_image_for_display(img_array):
    """Deprocesses image array for display (undoes scaling)."""
    # If you used standard preprocess_input (scales to -1 to 1 or similar):
    # This might need adjustment based on the exact preprocessing used.
    # A simple approach is to scale back to 0-255 assuming initial scaling.
    img_array = img_array.copy()
    if np.max(img_array) <= 1.0 and np.min(img_array) >= -1.0: # Heuristic check
        img_array = ((img_array + 1.0) * 127.5).astype(np.uint8) # Scale back from -1..1
    elif np.max(img_array) <= 1.0 and np.min(img_array) >= 0.0: # Heuristic check for 0..1
         img_array = (img_array * 255.0).astype(np.uint8)
    return img_array.squeeze() # Remove batch dim


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generates the Grad-CAM heatmap."""
    # Create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the top predicted class (or specified class)
    # for our input image with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Gradient of the output neuron (top predicted or chosen) w.r.t. the output feature map
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Vector of mean intensity of the gradient over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by "how important this channel is"
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization, normalize the heatmap between 0 & 1 and apply ReLU
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
def display_gradcam(img_path, heatmap, alpha=0.6, target_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"])):
    """Displays the heatmap superimposed on the original image."""
    # Load the original image (for display, not preprocessing)
    img = keras_image.load_img(img_path)
    img = keras_image.img_to_array(img)

    # Resize heatmap to match the original image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0])) # Use cv2 for convenience

    # Apply the heatmap (convert to RGB)
    heatmap = np.uint8(255 * heatmap)
    jet = cm.get_cmap("jet") # Get colormap
    jet_colors = jet(np.arange(256))[:, :3] # Use RGB part
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras_image.array_to_img(jet_heatmap)
    jet_heatmap = keras_image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras_image.array_to_img(superimposed_img)

    # Display
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()


In [ ]:
# Assuming 'model' is your trained Keras model object
# If not loaded, load it: model = tf.keras.models.load_model('your_model.h5')
model.summary()
# Look for the last Conv2D or similar layer name in the base model part.
# For EfficientNetB0, it's often 'top_conv'
# For ResNet50, it might be 'conv5_block3_out' or similar

In [ ]:
# --- Grad-CAM Explanation ---

# 1. Load your trained model (if not already loaded)
# model = tf.keras.models.load_model('path_to_your_model.h5')
# OR use the 'model' object if it's still in memory after training

# 2. !!! IMPORTANT: Set the name of the last convolutional layer !!!
# Use model.summary() to find the correct name for YOUR model architecture
LAST_CONV_LAYER_NAME = "top_conv" # Example for EfficientNetB0 - CHANGE THIS
# LAST_CONV_LAYER_NAME = "conv5_block3_out" # Example for ResNet50 - CHANGE THIS

# 3. Choose an image to explain
img_path = "/kaggle/input/sem-nffa-europe/data100/Fibres/L9_00fc0a86bd4f02995acdd5b3f63401b9.jpg" # Replace with an actual image path from your dataset

# 4. Preprocess the image for the model
img_array = preprocess_image(img_path, target_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

# 5. Get the model's prediction
preds = model.predict(img_array)
pred_index = np.argmax(preds[0])
pred_class_name = CONFIGURATION["CLASS_NAMES"][pred_index]
print(f"Predicted class: {pred_class_name} (Index: {pred_index}) with confidence {preds[0][pred_index]:.4f}")

# 6. Generate the Grad-CAM heatmap
heatmap = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, pred_index=pred_index)

# 7. Display the heatmap overlaid on the image
print("Displaying Grad-CAM heatmap...")
display_gradcam(img_path, heatmap, alpha=0.5) # Adjust alpha for heatmap intensity

# Optional: Explain a specific class (even if not the top prediction)
target_class_index = 1 # e.g., index for "Nanowires"
target_class_name = CONFIGURATION["CLASS_NAMES"][target_class_index]
print(f"\nGenerating heatmap for target class: {target_class_name}")
heatmap_target = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, pred_index=target_class_index)
display_gradcam(img_path, heatmap_target, alpha=0.5)

